In [2]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import numpy as np
import os
import pandas as pd
from pykonal import EikonalSolver, LinearInterpolator3D, GridND
import seispy

EARTH_RADIUS = 6371.
GOOGLE_DRIVE = os.environ['GOOGLE_DRIVE']
VMODEL_PATH  = os.path.join(GOOGLE_DRIVE, 'malcolm.white@usc.edu', 'data', 'velocity', 'White_et_al_2019a', 'White_et_al_2019a.regular.npz')
DB_PATH      = os.path.join(GOOGLE_DRIVE, 'malcolm.white@usc.edu', 'data', 'events', 'malcolmw', 'SJFZ_catalog_2008-2016.h5')

In [3]:
%matplotlib widget

Define convenience class to compute traveltimes using a source-centered coordinate system in the near field.

In [4]:
class TwoStageSolver(object):
    
    def __init__(self, coord_sys='spherical'):
        '''
        A convenience class to compute traveltimes using a
        source-centered coordinate system in the near field.
        '''
        self.coord_sys = coord_sys


    @property
    def near_field(self):
        '''
        The source-centered EikonalSolver for the near field.
        '''
        if not hasattr(self, '_near_field'):
            self._near_field = EikonalSolver(coord_sys='spherical')
        return (self._near_field)
    
    @property
    def far_field(self):
        '''
        The EikonalSolver for the far field.
        '''
        if not hasattr(self, '_far_field'):
            self._far_field = EikonalSolver(coord_sys=self.coord_sys)
        return (self._far_field)
    
    @property
    def src_loc(self):
        '''
        The coordinates of the source.
        '''
        return (self._src_loc)
    
    @src_loc.setter
    def src_loc(self, value):
        value = np.array(value)
        if np.any(value < self.far_field.vgrid.min_coords)\
                or np.any(value > self.far_field.vgrid.max_coords):
            raise(ValueError('Source location must lie inside far-field grid.'))
        self._src_loc = value

    def solve(self):
        self._init_near_field()
        self.near_field.solve()
        self.far_field.transfer_travel_times_from(
            self.near_field,
            self.src_loc  * [-1, 1, 1],
            set_alive=True
        )
        self.far_field.solve()
    
    def _init_near_field(self):
        if self.coord_sys == 'cartesian':
            drho = np.min(self.far_field.pgrid.min_coords) / 10
        else:
            drho = self.far_field.pgrid.node_intervals[0] / 10
        self.near_field.vgrid.min_coords     = drho, 0, 0
        self.near_field.vgrid.node_intervals = drho, np.pi/20, np.pi/20
        self.near_field.vgrid.npts           = 100, 21, 40
        self.near_field.transfer_velocity_from(self.far_field, self.src_loc)
        for it in range(self.near_field.vgrid.npts[1]):
            for ip in range(self.near_field.vgrid.npts[2]):
                idx = (0, it, ip)
                self.near_field.uu[idx]     = drho / self.near_field.vvp[idx]
                self.near_field.is_far[idx] = False
                self.near_field.close.push(*idx)

# Synthetic data

This example tests the location algorithm for the simplest and most ideal case: Homogeneous velocity structure and station coverage

## Generate synthetic data

### Define the computational grid

In [5]:
TAG            = 'homogeneous_0.0'
lat0, lon0, z0 = 44, 44, -5     # homogeneous_0.0
dlat, dlon, dz = 0.01, 0.01, 1 # homogeneous_0.0
nlat, nlon, nz = 201, 201, 26  # homogeneous_0.0
latmax         = lat0 + (nlat-1)*dlat
lonmax         = lon0 + (nlon-1)*dlon
zmax           = z0 + (nz-1)*dz

rho0, theta0, phi0 = EARTH_RADIUS-zmax, np.pi/2-np.radians(latmax), np.radians(lon0)
drho, dtheta, dphi = dz, np.radians(dlat), np.radians(dlon)
nrho, ntheta, nphi = nz, nlat, nlon

### Define the station locations

In [6]:
def init_stations(lat0, latmax, lon0, lonmax):
    stations = pd.DataFrame(columns=['sta_code', 'lat', 'lon', 'depth'])
    dlat, dlon = 0.25, 0.25 # homogeneous_0.0
    ilat = 0
    lat  = lat0
    while lat <= latmax:
        ilon, lon  = 0, lon0
        while lon <= lonmax:
            sta_code = f'{chr(ord("A") + ilat//26)}{chr(ord("A") + ilat%26)}{ilon:02d}'
            stations = stations.append(
                pd.DataFrame(
                    [[sta_code, lat, lon, 0]],
                    columns=['sta_code', 'lat', 'lon', 'depth']
                ),
                ignore_index=True
            )
            ilon += 1
            lon  += dlon
        ilat += 1
        lat  += dlat
    return(stations)

stations = init_stations(lat0, latmax, lon0, lonmax)

### Compute and store traveltime field for each station

In [ ]:
tt = dict()
for idx, station in stations.iloc[20:].iterrows():
    print(station['sta_code'])
    solver = TwoStageSolver(coord_sys='spherical')
    solver.far_field.vgrid.min_coords      = rho0, theta0, phi0
    solver.far_field.vgrid.node_intervals  = drho, dtheta, dphi
    solver.far_field.vgrid.npts            = nrho, ntheta, nphi
    solver.far_field.vv                    = np.ones(solver.far_field.vgrid.npts)
    solver.src_loc                         = seispy.coords.as_geographic(
        station[['lat', 'lon', 'depth']]
    ).to_spherical()
    solver.solve()
    break
#     tt[station['sta_code']] = solver_p.far_field.uu

### Save the traveltime fields

In [ ]:
np.savez(f'/Users/malcolmwhite/Desktop/ttlookup_{TAG}.npz', **tt)

### Define event locations

In [7]:
def init_events(lat0, latmax, lon0, lonmax, z0, zmax):
    events         = pd.DataFrame(columns=['lat', 'lon', 'depth', 'event_id'])
    lat_avg        = (lat0+latmax) / 2
    lon_avg        = (lon0+lonmax) / 2
    z_avg          = (z0+zmax) / 2
    dlat, dlon, dz = 0.1, 0.1, 1 # homogeneous_0.0
    ilat           = 0
    lat, lon, z    = lat0, lon0, z0
    while lat <= latmax:
        events = events.append(
                pd.DataFrame(
                    [[lat, lon_avg, z_avg, -1]],
                    columns=['lat', 'lon', 'depth', 'event_id']
                ),
                ignore_index=True
        )
        lat += dlat
    while lon <= lonmax:
        events = events.append(
                pd.DataFrame(
                    [[lat_avg, lon, z_avg, -1]],
                    columns=['lat', 'lon', 'depth', 'event_id']
                ),
                ignore_index=True
        )
        lon += dlon
    while z <= zmax:
        events = events.append(
                pd.DataFrame(
                    [[lat_avg, lon_avg, z, -1]],
                    columns=['lat', 'lon', 'depth', 'event_id']
                ),
                ignore_index=True
        )
        z += dz
    events['event_id'] = events.index.values
    return (events)

events = init_events(lat0, latmax, lon0, lonmax, z0, zmax)

### Compute synthetic travel times for events

In [ ]:
arrivals = pd.DataFrame(columns=['event_id', 'sta_code', 'tt', 'phase'])
for event_id, event in events.iterrows():
    print(f'Event #{event_id}')
    solver = TwoStageSolver(coord_sys='spherical')
    solver.far_field.vgrid.min_coords      = rho0, theta0, phi0
    solver.far_field.vgrid.node_intervals  = drho, dtheta, dphi
    solver.far_field.vgrid.npts            = nrho, ntheta, nphi
    solver.far_field.vv                    = np.ones(solver.far_field.vgrid.npts)
    solver.src_loc                         = seispy.coords.as_geographic(
        event[['lat', 'lon', 'depth']]
    ).to_spherical()
    solver.solve()
    uui = LinearInterpolator3D(solver.far_field.pgrid, solver.far_field.uu)
    for sta_idx, station in stations.iterrows():
        tt = uui(
            seispy.coords.as_geographic(
                station[['lat', 'lon', 'elev']] * [1, 1, -1]
            ).to_spherical()
        )
        arrivals = arrivals.append(
            pd.DataFrame(
                [[event_id, station['sta_code'], tt, 'P']],
                columns=['event_id', 'sta_code', 'tt', 'phase']
            ),
            ignore_index=True
        )

### Save event data

In [ ]:
with pd.HDFStore(f'/Users/malcolmwhite/Desktop/db_{TAG}.h5') as store:
    store['events'] = events
    store['arrivals'] = arrivals

### Create dictionary of traveltime field interpolators

In [ ]:
grid                 = GridND()
grid.min_coords      = rho0, theta0, phi0
grid.node_intervals  = drho, dtheta, dphi
grid.npts            = nrho, ntheta, nphi

tti = dict()

npz = np.load(os.path.join(GOOGLE_DRIVE, 'malcolm.white@usc.edu', 'proj', 'pykonal', 'data', 'locate', TAG, f'ttlookup_{TAG}.npz'))
for sta_code in npz.keys():
    tti[sta_code] = LinearInterpolator3D(grid, npz[sta_code])

### Define an objective function

In [ ]:
def objective(hypo, arrivals, tti):
    residuals = []
    for idx, arrival in arrivals.iterrows():
        residuals.append(tti[arrival['sta_code']]([6371, 1.58824962, 0]) - arrival['tt'])
    return (np.mean(np.abs(np.array(residuals))))

In [ ]:
objective(
#     seispy.coords.as_geographic(events.loc[0][['lat', 'lon', 'depth']]).to_spherical(),
    [ 6.36956586e+03,  1.58804003e+00, -2.00484042e-03],
    arrivals.set_index('event_id').loc[0],
    tti
)

In [ ]:
from  scipy.optimize import differential_evolution as optimize

In [ ]:
%%time
optimize(
    objective, 
    (
        (rho0, rho0+(nrho-1)*drho), 
        (theta0, theta0+(ntheta-1)*dtheta), 
        (phi0, phi0+(nphi-1)*dphi)
    ),
    args=(arrivals.set_index('event_id').loc[0], tti)
)

In [ ]:
seispy.coords.as_geographic(events.loc[0][['lat', 'lon', 'depth']]).to_spherical()

# START TEST

In [8]:
event   = events.iloc[52]
station = stations.iloc[47]
print(event, station)

lat         45
lon         45
depth        7
event_id    52
Name: 52, dtype: object sta_code     AF02
lat         45.25
lon          44.5
depth           0
Name: 47, dtype: object


In [9]:
# Event
event_solver = TwoStageSolver(coord_sys='spherical')
event_solver.far_field.vgrid.min_coords      = rho0, theta0, phi0
event_solver.far_field.vgrid.node_intervals  = drho, dtheta, dphi
event_solver.far_field.vgrid.npts            = nrho, ntheta, nphi
event_solver.far_field.vv                    = np.ones(event_solver.far_field.vgrid.npts)
event_solver.src_loc                         = seispy.coords.as_geographic(
    event[['lat', 'lon', 'depth']]
).to_spherical()
%time event_solver.solve()
uui_es = LinearInterpolator3D(event_solver.far_field.pgrid, event_solver.far_field.uu)

/Users/malcolmwhite/local/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in greater
/Users/malcolmwhite/local/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in less


CPU times: user 10.7 s, sys: 87.8 ms, total: 10.8 s
Wall time: 10.7 s


In [10]:
station_solver = TwoStageSolver(coord_sys='spherical')
station_solver.far_field.vgrid.min_coords      = rho0, theta0, phi0
station_solver.far_field.vgrid.node_intervals  = drho, dtheta, dphi
station_solver.far_field.vgrid.npts            = nrho, ntheta, nphi
station_solver.far_field.vv                    = np.ones(station_solver.far_field.vgrid.npts)
station_solver.src_loc                         = seispy.coords.as_geographic(
    station[['lat', 'lon', 'depth']]
).to_spherical()
%time station_solver.solve()
uui_se = LinearInterpolator3D(station_solver.far_field.pgrid, station_solver.far_field.uu)

/Users/malcolmwhite/local/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in greater
/Users/malcolmwhite/local/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in less


CPU times: user 11.3 s, sys: 148 ms, total: 11.5 s
Wall time: 11.4 s


In [11]:
(
    uui_es(seispy.coords.as_geographic(station[['lat', 'lon', 'depth']]).to_spherical()),
    uui_se(seispy.coords.as_geographic(event[['lat', 'lon', 'depth']]).to_spherical())
)

(48.53406473573391, 48.533038860332546)

In [ ]:
np.any(station_solver.far_field.uu == event_solver.far_field.uu)

In [ ]:
# solver = event_solver
nodes   = solver.near_field.pgrid.nodes
x0      = solver.src_loc[0] * np.sin(solver.src_loc[1]) * np.cos(solver.src_loc[2])
y0      = solver.src_loc[0] * np.sin(solver.src_loc[1]) * np.sin(solver.src_loc[2])
z0      = solver.src_loc[0] * np.cos(solver.src_loc[1])
xx_near = nodes[...,0] * np.sin(nodes[...,1]) * np.cos(nodes[...,2]) + x0
yy_near = nodes[...,0] * np.sin(nodes[...,1]) * np.sin(nodes[...,2]) + y0
zz_near = nodes[...,0] * np.cos(nodes[...,1]) + z0

nodes   = solver.far_field.pgrid.nodes
xx_far  = nodes[...,0] * np.sin(nodes[...,1]) * np.cos(nodes[...,2])
yy_far  = nodes[...,0] * np.sin(nodes[...,1]) * np.sin(nodes[...,2])
zz_far  = nodes[...,0] * np.cos(nodes[...,1])


plt.close('all')
fig = plt.figure()
ax  = fig.add_subplot(1, 2, 1, projection='3d')
# i1, i2, i3 = slice(None, None, 4), slice(None, None, 2), slice(None, None, 2)
i1, i2, i3 = slice(None), slice(None), slice(None)
ax.scatter(
    xx_far[i1,i2,i3].flatten(), 
    yy_far[i1,i2,i3].flatten(),
    zz_far[i1,i2,i3].flatten(),
    c=solver.far_field.uu[i1,i2,i3].flatten(),
#     xx_far[idxs].flatten(), 
#     yy_far[idxs].flatten(),
#     zz_far[idxs].flatten(),
#     c=solver.far_field.uu[idxs].flatten(),
    cmap=plt.get_cmap('jet_r'),
    s=1
)

ax  = fig.add_subplot(1, 2, 2, projection='3d')
i1, i2, i3 = slice(None,None,2), slice(None), slice(None)
ax.scatter(
    xx_near[i1,i2,i3].flatten(), 
    yy_near[i1,i2,i3].flatten(),
    zz_near[i1,i2,i3].flatten(),
    c=solver.near_field.uu[i1,i2,i3].flatten(),
    cmap=plt.get_cmap('jet_r'),
#     vmin=solver.far_field.uu.min(),
#     vmax=solver.far_field.uu[~np.isinf(solver.far_field.uu)].max(),
    s=1
)

# END TEST

### Plot traveltime field

In [ ]:
nodes   = solver_p.near_field.pgrid.mesh
xx_near = nodes[...,0] * np.sin(nodes[...,1]) * np.cos(nodes[...,2])
yy_near = nodes[...,0] * np.sin(nodes[...,1]) * np.sin(nodes[...,2])
zz_near = nodes[...,0] * np.cos(nodes[...,1])

nodes   = solver_p.far_field.pgrid.mesh
xx_far  = nodes[...,0] * np.sin(nodes[...,1]) * np.cos(nodes[...,2])
yy_far  = nodes[...,0] * np.sin(nodes[...,1]) * np.sin(nodes[...,2])
zz_far  = nodes[...,0] * np.cos(nodes[...,1])


plt.close('all')
fig = plt.figure()
ax  = fig.add_subplot(1, 1, 1, projection='3d')
# ax.scatter(
#     xx_far.flatten(), yy_far.flatten(), zz_far.flatten(),
#     c=solver_p.far_field.uu.flatten(),
#     cmap=plt.get_cmap('jet_r')
# )
ax.scatter(
    xx_near.flatten(), yy_near.flatten(), zz_near.flatten(),
    c=solver_p.near_field.uu.flatten(),
    cmap=plt.get_cmap('jet_r'),
    s=1
)

# Real data

Read velocity model

In [ ]:
vmod = seispy.velocity.VelocityModel(VMODEL_PATH, fmt='npz')

Read network geometry

In [ ]:
db = seispy.pandas.catalog.Catalog(DB_PATH, fmt='hdf5', tables=['site'])

In [ ]:
solver_p = TwoStageSolver()
solver_p.far_field.vgrid.min_coords      = vmod.rho0, vmod.theta0, vmod.phi0
solver_p.far_field.vgrid.node_intervals  = vmod.drho, vmod.dtheta, vmod.dphi
solver_p.far_field.vgrid.npts            = vmod.nrho, vmod.ntheta, vmod.nphi
solver_p.far_field.vv                    = vmod._vp

In [ ]:
%time solver_p.solve()

In [ ]:
np.savez('/Users/malcolmwhite/Desktop/test.npz', uu=solver_p.far_field.uu)

In [ ]:
ir, it, ip = slice(None), slice(None), 0

In [ ]:
ir, it, ip = slice(None), slice(None), 0

nodes = solver_p.far_field.pgrid.mesh
nodes_x = nodes[...,0] * np.sin(nodes[...,1]) * np.cos(nodes[...,2])
nodes_y = nodes[...,0] * np.sin(nodes[...,1]) * np.sin(nodes[...,2])
nodes_z = nodes[...,0] * np.cos(nodes[...,1])

fig = plt.figure()
ax  = fig.add_subplot(1, 1, 1)
ax.pcolormesh(
    nodes_y[ir, it, ip],
    nodes_z[ir, it, ip],
    solver_p.far_field.uu[ir, it, ip],
    cmap=plt.get_cmap('jet_r'),
    shading='gouraud'
)

In [ ]:
solver_p.far_field.uu[ir, it, ip].shape

In [ ]:
%time solver_p.near_field.solve()

In [ ]:
solver_p.near_field.uu

In [ ]:
solver_p.src_loc = 6365, 0.98, -2.03

In [ ]:
solver_p.src_loc